# High resolution vertical coordinates for double-gyre simulation
To set up the initialization files for the spin up with COBALT, run the physics only spin up for one more year with a higher resolution vertical axis. Use the vertical axis in the restart file, ignoring actual layers.

### Steps
1. Create a new simulation folder.
2. Move the appropriate restart file into the RESTART folder.
3. In `tigress`, access navigate to the simulation's `./INPUT/code/` and run this Notebook to create the vertical coordinate axis.
4. Change `DIAG_COORD_DEF_Z = "WOA09"` to `DIAG_COORD_DEF_Z = FILE:vgrid_HighRes4000m_surface2m.nc,dz`


In [ ]:
import numpy as np
import xarray as xr
import netCDF4 as nc
from datetime import datetime

In [ ]:
# Manual input
fillval   = -1e+20  # set fill value (Raphael/Enhui: -1e+20; Elizabeth:-1.e+34)

# Paths 
rpath     = '../../RESTART/'  # to restart file
ipath     = '../'  # save into input

# Restart year
Ryr       = '2490'

# Max ocean depth (flat bottom)
strdepth  = '4000m'

In [ ]:
### Create a new vertical axis that has the same resolution has the restart files  ###
# Load MOM restart file 
dsR     = xr.open_dataset(rpath + 'MOM.res_Y' + Ryr + '_D001_S00000.nc')

# From Elizabeth's MOM6 Example
writing = nc.Dataset(ipath + "vgrid_HighRes" + strdepth + "_surface2m.nc","w",format="NETCDF4_CLASSIC")
dz      = writing.createDimension("dz", len(dsR.Layer))
zw      = writing.createDimension("zw", len(dsR.Interface))
zt      = writing.createDimension("zt", len(dsR.Layer))

dz      = writing.createVariable("dz","f8",("dz"))
dz[:]   = np.diff(dsR.Interface)
dz.long_name = "Diagnostic depth thickness in the vertical coordinate"
dz.comment   = "Used for diagnostics only according to Jingru requirement"
dz.units     = "m"

zw      = writing.createVariable("zw","f8",("zw"))
zw[:]   = dsR.Interface.values
zw.long_name = "Diagnostic interface depth in the vertical coordinate"
zw.comment   = "Used for diagnostics only according to Jingru requirement"
zw.units     = "m"

zt      = writing.createVariable("zt","f8",("zt"))
zt[:]   = dsR.Layer.values
zt.long_name = "Diagnostic tracer point depth in the vertical coordinate"
zt.comment   = "Used for diagnostics only according to Jingru requirement"
zt.units     = "m"

writing.creator = "JCG in LRG at Princeton University"
writing.creation_date = datetime.now().strftime("%d-%b-%Y %H:%M:%S")

writing.close()